In [9]:
import pandas as pd
import numpy as np

file_name1 = 'SSMI_swizerland.csv'
file_name2 = 'TWII.csv'

# Read csv / excel files
df1 = pd.read_csv(file_name1)[['Date', 'Close']]
df2 = pd.read_csv(file_name2)[['Date', 'Close']]

# For excel files:
# df = pd.read_excel(file_name, usecols=['Date', 'Close'])

df1['Date'] = pd.to_datetime(df1['Date'])
df1['Close'] = pd.to_numeric(df1['Close'])

df2['Date'] = pd.to_datetime(df2['Date'])
df2['Close'] = pd.to_numeric(df2['Close'])

# Drop null / NaN values
df1.dropna(inplace=True)
df2.dropna(inplace=True)

# Sort data from old to new
df1.sort_values(by='Date', ascending=True, inplace=True)
df2.sort_values(by='Date', ascending=True, inplace=True)

df2

,Date,Close
0,2000-01-04,8756.549805
1,2000-01-05,8849.870117
2,2000-01-06,8922.030273
3,2000-01-07,8849.870117
4,2000-01-10,9102.599609
...,...,...
4771,2018-12-24,9639.700195
4772,2018-12-25,9527.089844
4773,2018-12-26,9478.990234
4774,2018-12-27,9641.559570


In [10]:
# Calculate Log Return of close values
df1['Rt'] = np.log(df1['Close'] / df1['Close'].shift(1))
df2['Rt'] = np.log(df2['Close'] / df2['Close'].shift(1))

df1.dropna(inplace=True)
df2.dropna(inplace=True)

In [11]:
bin_no = 12

# Function to calculate Symbol / Bin_Number
def which_bin(val, min, max):
    L = min - 1
    for i in range (1, bin_no):
        R = min + (max - min) / bin_no * i
        if  L < val <= R:
            return i

        L = R

    return bin_no

In [12]:
# Assign Symbol / Bin_number to each return value
min1 = df1['Rt'].min() 
max1 = df1['Rt'].max()

min2 = df2['Rt'].min() 
max2 = df2['Rt'].max()

df1['Symbol'] = df1['Rt'].apply(lambda value : which_bin(value, min1, max1)) # apply function to each element of the column
df2['Symbol'] = df2['Rt'].apply(lambda value : which_bin(value, min2, max2))

In [13]:
# Merge two dataframe based on the same date
df = pd.merge(df1, df2, on='Date', how='inner')

df

,Date,Close_x,Rt_x,Symbol_x,Close_y,Rt_y,Symbol_y
0,2000-01-05,7181.299805,-0.012015,5,8849.870117,0.010601,9
1,2000-01-06,7280.600098,0.013733,7,8922.030273,0.008121,8
2,2000-01-07,7448.000000,0.022732,7,8849.870117,-0.008121,7
3,2000-01-10,7452.799805,0.000644,6,9102.599609,0.028157,10
4,2000-01-11,7420.100098,-0.004397,6,8927.030273,-0.019476,6
...,...,...,...,...,...,...,...
4535,2018-12-19,8540.160156,0.001393,6,9783.209961,0.006603,8
4536,2018-12-20,8414.480469,-0.014826,5,9674.519531,-0.011172,7
4537,2018-12-21,8417.290039,0.000334,6,9676.669922,0.000222,8
4538,2018-12-27,8195.639648,-0.026686,4,9641.559570,0.017005,9


In [14]:
# Frequency Calculation
n = bin_no + 1

Fxy = [[0] * n for _ in range (n)]
Fx = [0] * n
Fy = [0] * n

for row in df.itertuples():
    x = row.Symbol_x
    y = row.Symbol_y

    Fxy[x][y] += 1
    Fx[x] += 1
    Fy[y] += 1

In [15]:
# Probability Calculation
Pxy = [[0] * n for _ in range (n)]
Px = [0] * n
Py = [0] * n

total_frequency = sum(Fx)

for x in range (1, n):
    for y in range (1, n):
        Pxy[x][y] = Fxy[x][y] / total_frequency

for i in range (1, n):
    Px[i] = Fx[i] / total_frequency
    Py[i] = Fy[i] / total_frequency

In [16]:
# Calculate mutual information I(X, Y)
import math

Ixy = 0

for i in range (1, n):
    for j in range (1, n):
        if Pxy[i][j] > 0 and Px[i] * Py[j] > 0:
            Ixy += Pxy[i][j] * math.log2(Pxy[i][j] / (Px[i] * Py[j]))

print(f"Mututal Information of ({file_name1}, {file_name2}) is = {Ixy}")

Mututal Information of (SSMI_swizerland.csv, TWII.csv) is = 0.06335505391398863
